# Reconstruction (without truncations)
_Author: Lucas Z. Brito_

This notebook reconstructs the Haldane Shastry Hamiltonian. 

### Reconstruction and model parameters

In [25]:
n = 21 # Chain size 
atol = 1e-2 # Absolute error for sanity checks 
output_dir = "../data/hs_data/"

"../data/hs_data/"

In [26]:
using JLD
using DataFrames
using CSV

### Reconstruction

In [27]:
include("./src/hs_tools.jl")

# Make wavefunction and correlation matrix and save to a directory to avoid 
# running calculation again. 
# make_wf_cm(n, output_dir) # Can take roughly a minute for n > 10. 

ft_mat

In [28]:
cm_wf = load(output_dir * "cm_wf_n=$n.jld")
k = cm_wf["wf"]
cm = cm_wf["cm"]

# Haldane shastry coefficients
hs = normalize(hs_coeffs(n))
hs

10-element Vector{Float64}:
 0.9576499897788423
 0.24485153069866578
 0.11300013198803671
 0.06703710767239983
 0.045981936502955886
 0.03480161435528048
 0.02836380138986998
 0.024549583758207708
 0.02238106199590267
 0.021392318302687525

In [34]:
# Make correlation matrix
# cm = make_corr_mat(n, k)

printstyled("Correlation matrix: ", bold=true); flush(stdout)
display(cm)

# Diagonalize correlation matrix
eig = eigen(cm)

printstyled("Eigenvectors:", bold=true); flush(stdout)
display(eig.vectors)
printstyled("Eigenvalues:", bold=true); flush(stdout)
display(eig.values)

Correlation matrix: 

10×10 Matrix{Float64}:
  0.000105852  -0.00075804    0.000982603  …   9.73075e-5   -3.04e-5
 -0.00075804    0.00565397   -0.00785232      -0.000646765   0.000209349
  0.000982603  -0.00785232    0.0126316        0.00102596   -0.000329301
 -0.000615588   0.00492544   -0.0094629       -0.00150226    0.000480461
  0.000470139  -0.00333966    0.00603574       0.0021262    -0.000668826
 -0.000334716   0.00236379   -0.00398135   …  -0.00310966    0.000967473
  0.000246625  -0.00167261    0.0027371        0.0048687    -0.00144025
 -0.000163782   0.00111685   -0.00178712      -0.00827655    0.00243408
  9.73075e-5   -0.000646765   0.00102596       0.0097195    -0.00430244
 -3.04e-5       0.000209349  -0.000329301     -0.00430244    0.00267986

Eigenvectors:

10×10 Matrix{Float64}:
 0.0694787   0.97186     0.159771   …  -0.0389301   0.0339759   0.0248761
 0.279489    0.179777   -0.404979       0.318322   -0.27541    -0.186352
 0.318336    0.0332595  -0.439314      -0.45566     0.458349    0.314918
 0.331878   -0.017816   -0.340725       0.0492571  -0.374148   -0.386625
 0.338081   -0.0412131  -0.209004       0.367213    0.202339    0.448151
 0.341375   -0.053637   -0.0262217  …  -0.431388    0.0948351  -0.441139
 0.343272   -0.0607909   0.12895        0.167851   -0.358262    0.409413
 0.344396   -0.0650294   0.28915        0.271012    0.463799   -0.321925
 0.345035   -0.0674391   0.389769      -0.465796   -0.396051    0.210209
 0.345326   -0.0685378   0.452603       0.218119    0.150573   -0.0715251

Eigenvalues:

10-element Vector{Float64}:
 5.597155483673694e-17
 4.112156177434893e-15
 0.0001320969630435512
 0.0005502731341263991
 0.001383283397916225
 0.0028589158059040757
 0.0066238521130070034
 0.011851740930315715
 0.02204544932218035
 0.04928470613871315

In [35]:
# Check that conserved quantity and Hamiltonian are in nullspace of CM
nullsp = eig.vectors[:, 1:2]

h_proj, h_is_in_nsp = in_subspace(nullsp, hs, atol)
# in_subspace([eig.vectors[:, 1]], hs, atol)

conserved_quant = normalize(ones(trunc(Int, n / 2)))
stot_proj, stot_is_in_nsp = in_subspace(nullsp, conserved_quant, atol)

# Project HS coefficient onto nullspace; this will be our "reconstructed 
# Hamiltonian."
coeffs_recon = nullsp * inv(nullsp' * nullsp) * nullsp' * hs

if h_is_in_nsp
	printstyled("Hamiltonian is in nullspace ✓", bold=true, color=:green)
else 
	printstyled("Hamiltonian is not in nullspace ×", bold=true, color=:red)
end 

println()

if stot_is_in_nsp
	printstyled("S_tot is in nullspace ✓", bold=true, color=:green)
else 
	printstyled("S_tot is not in nullspace ×", bold=true, color=:red)
end 

println()
println("(atol = $atol)")

Hamiltonian is in nullspace ✓
S_tot is in nullspace ✓
(atol = 0.01)


In [36]:
printstyled("Projections onto nullspace:\n", bold=true)
print("H:\t")
print(h_proj)
println()
print("S_tot:\t")
print(stot_proj)
println()

Projections onto nullspace:
H:	1.0000000000000002
S_tot:	1.0000000000000004


In [37]:
# Save data 
begin
	df = DataFrame(r=1:(trunc(Int, n / 2)), 
				   n=n,
				   coeffs_anal=hs, 
				   coeffs_recon=real.(coeffs_recon))
	fn = CSV.write(output_dir * "recon_n=$n.csv", df)
	printstyled("Written reconstruction data to $fn.", bold=true, color=:green)
end

Written reconstruction data to ../data/hs_data/recon_n=21.csv.